In [4]:
import os
import boto3
import sagemaker
from sentence_transformers import SentenceTransformer, util

role = 'arn:aws:iam::484906661071:role/sagemaker'

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/sentence-transformers/multi-qa-mpnet-base-dot-v1"

In [ ]:
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model.save_pretrained("./model")

In [1]:
!ls -rtlh ./model/

total 419M
-rw-rw-r-- 1 josh josh  116 Nov 11 14:01 config_sentence_transformers.json
-rw-rw-r-- 1 josh josh  669 Nov 11 14:01 config.json
-rw-rw-r-- 1 josh josh  438 Nov 11 14:01 tokenizer_config.json
-rw-rw-r-- 1 josh josh  239 Nov 11 14:01 special_tokens_map.json
-rw-rw-r-- 1 josh josh 418M Nov 11 14:01 pytorch_model.bin
-rw-rw-r-- 1 josh josh 227K Nov 11 14:01 vocab.txt
-rw-rw-r-- 1 josh josh 456K Nov 11 14:01 tokenizer.json
-rw-rw-r-- 1 josh josh   53 Nov 11 14:01 sentence_bert_config.json
drwxrwxr-x 2 josh josh 4.0K Nov 11 14:01 1_Pooling
-rw-rw-r-- 1 josh josh 8.2K Nov 11 14:01 README.md
-rw-rw-r-- 1 josh josh  229 Nov 11 14:01 modules.json


In [2]:
!cd model && tar czvf ../model.tar.gz *

1_Pooling/
1_Pooling/config.json
config.json
config_sentence_transformers.json
modules.json
pytorch_model.bin
README.md
sentence_bert_config.json
special_tokens_map.json
tokenizer_config.json
tokenizer.json
vocab.txt


In [5]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)
print(os.path.join(bucket, key))

sagemaker-us-east-1-484906661071/sagemaker/sentence-transformers/multi-qa-mpnet-base-dot-v1/model.tar.gz


In [6]:
pretrained_model_data = "s3://{}/{}".format(bucket, key)
pretrained_model_data

's3://sagemaker-us-east-1-484906661071/sagemaker/sentence-transformers/multi-qa-mpnet-base-dot-v1/model.tar.gz'

In [7]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import RealTimePredictor
from sagemaker import get_execution_role

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

In [23]:
pytorch_model = PyTorchModel(model_data = pretrained_model_data, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code', 
                             framework_version = '1.3.1',
                             py_version = 'py3',
                             predictor_cls=StringPredictor)
# instance_type='local'
predictor = pytorch_model.deploy(instance_type='local', initial_instance_count=1)


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/josh/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Attaching to oka7e72le0-algo-1-w90ig
oka7e72le0-algo-1-w90ig | Collecting sentence-transformers
oka7e72le0-algo-1-w90ig |   Downloading https://files.pythonhosted.org/packages/3a/45/6e569fd41f539300903bc22dc10c0220694a1b13a0b57c9bf50a3d5c9cdf/sentence-transformers-2.1.0.tar.gz (78kB)
     |████████████████████████████████| 81kB 1.8MB/s 
oka7e72le0-algo-1-w90ig | Collecting numpy>=1.17
oka7e72le0-algo-1-w90ig |   Downloading https://files.pythonhosted.org/packages/14/32/d3fa649ad7ec0b82737b92fefd3c4dd376b0bb23730715124569f38f3a08/numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8MB)
     |████████████████████████████████| 14.8MB 328kB/s 
oka7e72le0-algo-1-w90ig | Collecting transformers<5.0.0,>=4.6.0
oka7e72le0-algo-1-w90ig |   Downloading https://files.pythonhosted.org/packages/2b/db/9145468493530a60046e59fc2a72b974591e38c1f3fc1b6c75112a91e492/transformers-4.12.3-py3-none-any.whl (3.1MB)
     |████████████████████████████████| 3.1MB 1.3MB/s 
oka7e72le0-algo-1-w90ig | Collecting tok

RuntimeError: Giving up, endpoint didn't launch correctly

In [24]:
import json

payload = str('denim dress')
features = predictor.predict(payload)
embading = json.loads(features)

len(embading)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "Requested unsupported ContentType in content_type: application/octet-stream". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2021-11-12-01-18-29-802 in account 484906661071 for more information.